## _Natural Language Processing(NLP)_
우리의 말을 컴퓨터에게 이해시키기 위한 기술

단어는 <b>의미의 최소 단위</b>
* 시소러스를 활용한 기법
* 통계 기반 기법
* 추론 기반 기법
### 시소러스
유의어 사전(뜻이 같은 단어)
모든 단어에 대한 유의어 집합을 만든 후, 단어들의 관계를 그래프로 표현하여 단어 사이 연결을 정의
<br/>
### 한계
* 시대 변화에 대응하기 어렵다
* 비용이 많이 든다
* 단어의 미묘한 차이를 표현 할 수 없다
### 통계 기반 기법
corpus : 자연어 처리나 애플리케이션을 염두에 두고 수집된 대량의 텍스트 데이터

In [1]:
# 파이썬으로 말뭉치 전처리 하기
text = 'You say goodbye and i say hello.'

text = text.lower()
text = text.replace('.', ' .')
text

words = text.split(' ')
words


['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [2]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [3]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [4]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
            
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [5]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

### 단어의 분산 표현
색을 벡터로 표현하듯(RGB) 단어를 벡터로 표현하는 방법
단어를 고정 길이의 밀집벡터(dense vector)로 표현 

### 분포 가설
단어의 의미는 주변 단어에 의해 형성된다 
<b>단어 자체에는 의미가 없고 그 단어가 사용된 맥락이 의미를 결정한다</b>
<br/>
맥락 : 특정 단어를 중심에 둔 그 주변 단어 

In [6]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess
text = 'You say goodBye and i say hello.'
corpus, word_to_id, id_to = preprocess(text)

print(corpus)
print(id_to_word)


[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [7]:
# 동시 발생 행렬
C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0],
], dtype=np.int32)

In [8]:
print(C[word_to_id['goodbye']])


[0 1 0 1 0 0 0]


In [9]:
#동시 발생 행렬 자동화
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i 
            right_idx = idx + 1
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    return co_matrix

### 벡터간 유사도 
cosine simularity : 두 벡터가 가리키는 방향이 얼마나 비슷한가 

In [10]:
def cos_similarity(x,y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

In [11]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity

text = 'You say goodBye and i say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0, c1))

0.7071067691154799


In [12]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return 
    
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity))
        
        count += 1
        if count >= top:
            return 
        
    

### 통계 기반 기법 개선하기

점별 상호정보량(PMI) 
$PMI(x,y) = log_2\frac{P(x,y)}{P(x)P(y)} = log_2\frac{C(x,y) * N}{C(x)C(y)}$
두 단어의 동시 발생 횟수가 0이면 -$\infty$

그래서 양의 상호정보량(PPMI) 사용
$PPMI(x,y) = max(0, PMI(x,y))$

PPMI 행렬 : 말뭉치의 어휘 수가 증가함에 따라 각 단어 벡터의 차원 수도 증가한다


In [13]:
def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i,j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print('%.1f %% 완료' % (100*cnt/total))
    
    return M

In [14]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess, create_co_matrix, cos_similarity, ppmi

text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3)
print('동시발생 행렬')
print(C)
print('-'*50)
print('PPMI')
print(W)

동시발생 행렬
[[0 1 0 0 0 0]
 [1 0 1 0 1 1]
 [0 1 0 1 0 0]
 [0 0 1 0 1 0]
 [0 1 0 1 0 0]
 [0 1 0 0 0 0]]
--------------------------------------------------
PPMI
[[0.    1.585 0.    0.    0.    0.   ]
 [1.585 0.    0.585 0.    0.585 1.585]
 [0.    0.585 0.    1.585 0.    0.   ]
 [0.    0.    1.585 0.    1.585 0.   ]
 [0.    0.585 0.    1.585 0.    0.   ]
 [0.    1.585 0.    0.    0.    0.   ]]


## 특이값 분해
$ \mathbf{X = USV^T} $
<br/><br/>
U와 V는 직교 행렬이고 열벡터는 서로 직교   <br/>
S는 대각행렬이고 대각성분은 특이값이 큰 순서대로 나열 되어 있음(특이값은 중요도)


In [16]:
import sys 
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from common.util import preprocess,create_co_matrix, ppmi

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(id_to_word)
C = create_co_matrix(corpus, vocab_size, window_size=1)
W = ppmi(C)

U,S,V = np.linalg.svd(W)